# Load Modules and File Paths

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from os.path import abspath
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
matplotlib.rcParams['text.latex.unicode']=False
import datetime
from operator import add
import pickle
from sklearn.impute import SimpleImputer as Imputer
from osgeo import gdal
import xarray as xr
import numpy as np
import richdem as rd
import joblib
filename = 'finalized_model.sav'


imp = Imputer(strategy='mean')

# --------------------- #
# Lets Rips Some Treads #
# --------------------- #

from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()


if abspath('../../utils/') not in sys.path:
    sys.path.append(abspath('../../utils/'))
from utils import GetExtent, ventana

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  if __name__ == '__main__':


In [2]:
SITENAME = 'tuolumne'
name = 'snowdepth'
formato = 'netcdf'
YEAR = '2015'
path_to_data = f'/srv/shared/deep_stac/data/{name}/{formato}/{YEAR}'
path_to_topo = '/srv/shared/deep_stac/data/topo.nc'
files = os.listdir(path_to_data)
files = np.sort([f for f in files if 'int' in f])
fname_list = [os.path.join(path_to_data, f) for f in np.sort(files)]
dates = [d.split('_')[0] for d in files]
dates = np.sort(dates).tolist()
# dates = [pd.to_datetime(d.split('_')[0]).date() for d in files]

### Read in Lidar and Topo Data

In [3]:
# --------------- #
# Read Lidar Data #
# --------------- #
ds = xr.open_mfdataset(fname_list, concat_dim='flight_date', chunks={'x':1000, 'y':1000})
ds.close()

dtopo = xr.open_dataset(path_to_topo, chunks={'x':1000, 'y':1000})
dtopo.close()

/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied do not have global dimension coordinates. In
future, to continue concatenating without supplying dimension
coordinates, please use the new `combine_nested` function (or the
`combine='nested'` option to open_mfdataset.
  coords=coords)


### Sub select features to include to the dem. 

In [4]:
add_to_lidar = ['dem','veg_height','mask']
dtopo = dtopo[add_to_lidar]

In [5]:
# -------------------- #
# Remove This features #
# -------------------- #

ds = ds.drop('transverse_mercator')

dtopo = dtopo.expand_dims('flight_date')

### Print Data Details

In [6]:
print('TOPO: \n\n',dtopo, '\n\n\nLidar:\n\n', ds)

TOPO: 

 <xarray.Dataset>
Dimensions:     (flight_date: 1, x: 17569, y: 17002)
Coordinates:
  * x           (x) float32 254007.0 254010.0 254013.0 ... 306708.0 306711.0
  * y           (y) float32 4230327.0 4230324.0 ... 4179327.0 4179324.0
Dimensions without coordinates: flight_date
Data variables:
    dem         (flight_date, y, x) float32 dask.array<shape=(1, 17002, 17569), chunksize=(1, 1000, 1000)>
    veg_height  (flight_date, y, x) float32 dask.array<shape=(1, 17002, 17569), chunksize=(1, 1000, 1000)>
    mask        (flight_date, y, x) uint8 dask.array<shape=(1, 17002, 17569), chunksize=(1, 1000, 1000)>
Attributes:
    last_modified:       [2019-08-08 17:17:16] Data added or updated
    Conventions:         CF-1.6
    dateCreated:         2019-08-08 17:17:34
    Title:               Topographic Images for SMRF/AWSM
    history:             [2019-08-08 17:17:34] Create netCDF4 file using Basi...
    institution:         USDA Agricultural Research Service, Northwest Waters...
  

In [7]:
ds['date'] = (['date'], dates)
ds = ds.assign_coords(date=ds.date)

In [8]:
dates

['20150218',
 '20150306',
 '20150325',
 '20150403',
 '20150409',
 '20150415',
 '20150427',
 '20150501',
 '20150528',
 '20150608']

In [9]:
datos = {}

In [10]:
datos['snowdepth'] = np.divide(ds.Band1[0,:,:].values,100)
datos['vegh'] = dtopo.veg_height.values.squeeze(axis=0)
datos['dem'] = dtopo.dem.values.squeeze(axis=0)

In [11]:
print(type(datos['dem']))
print(datos['dem'].shape)

<class 'numpy.ndarray'>
(17002, 17569)


## Generate Topographic Features from DEM

In [12]:
datos['dem'] = rd.rdarray(datos['dem'], no_data=-9999)

datos['slope'] = rd.TerrainAttribute(
    rd.rdarray(datos['dem'], no_data=-9999),
    attrib='slope_riserun'
)
datos['aspect'] = rd.TerrainAttribute(
    rd.rdarray(datos['dem'], no_data=-9999), 
    attrib='aspect')

datos['curvature'] = rd.TerrainAttribute(
    rd.rdarray(datos['dem'], no_data=-9999), 
    attrib='profile_curvature'
)

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)


### Visualize Features

In [ ]:
feat_to_viz = 'aspect'

plt.figure(figsize=(10,15))
ax = plt.subplot(1,1,1)
im = plt.imshow(
    np.flipud(datos[feat_to_viz]),
    origin='lower', 
    cmap='jet'
#     vmin=0, vmax=360
)
# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)

### Stack all the Input Features

In [13]:
features = ['dem', 'vegh', 'slope', 'aspect', 'curvature']
for f in features:
    datos[f] = np.expand_dims(np.array(datos[f]), axis=-1)
    

In [14]:
for llave, d in datos.items():
    print(llave, d.shape)

snowdepth (17002, 17569)
vegh (17002, 17569, 1)
dem (17002, 17569, 1)
slope (17002, 17569, 1)
aspect (17002, 17569, 1)
curvature (17002, 17569, 1)


In [17]:
X = np.stack([d for llave, d in datos.items() if llave is not 'snowdepth'], axis=-1)
Y = datos['snowdepth']

In [15]:
joblib.dump(datos, '../../processed/datos.joblib')  

['../../processed/datos.joblib']

In [42]:
Xlist = ventana(signal=X, kernel_size=(1000, 1000), stride=(1000, 1000))

In [ ]:
Ylist = ventana(signal=Y, kernel_size)